In [3]:
pip install numpy opencv-python


In [4]:
import numpy as np
import cv2
from collections import defaultdict


# Define color ranges (in HSV)
COLOR_RANGES = {
    'red': ([0, 100, 100], [10, 255, 255]),
    'blue': ([110, 100, 100], [130, 255, 255]),
    'green': ([50, 100, 100], [70, 255, 255]),
    'yellow': ([20, 100, 100], [30, 255, 255])
}

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    frame_count = 0
    ball_trackers = defaultdict(lambda: defaultdict(dict))
    events = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        balls = detect_balls(frame)
        update_trackers(ball_trackers, balls, frame_count)
        new_events = check_events(ball_trackers, frame_count)
        events.extend(new_events)

    cap.release()
    return events



In [5]:
def detect_balls(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    balls = {}

    for color, (lower, upper) in COLOR_RANGES.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            if cv2.contourArea(contour) > 100:  # Minimum area threshold
                M = cv2.moments(contour)
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                balls[center] = color

    return balls

In [6]:
def update_trackers(trackers, balls, frame_count):
    for center, color in balls.items():
        if color not in trackers or all(abs(existing_center[0] - center[0]) > 20 or abs(existing_center[1] - center[1]) > 20 for existing_center in trackers[color]):
            trackers[color][center] = {'first_seen': frame_count, 'last_seen': frame_count, 'current_quadrant': get_quadrant(center)}
        else:
            closest_center = min(trackers[color], key=lambda x: ((x[0] - center[0])**2 + (x[1] - center[1])**2)**0.5)
            trackers[color][closest_center]['last_seen'] = frame_count
            trackers[color][closest_center]['current_quadrant'] = get_quadrant(center)
            if closest_center != center:
                trackers[color][center] = trackers[color].pop(closest_center)

def get_quadrant(center):
    x, y = center
    if x < 320 and y < 240:
        return 1
    elif x >= 320 and y < 240:
        return 2
    elif x < 320 and y >= 240:
        return 3
    else:
        return 4

In [7]:
def check_events(trackers, frame_count):
    events = []
    for color in list(trackers.keys()):
        for center in list(trackers[color].keys()):
            data = trackers[color][center]
            if frame_count - data['last_seen'] > 5:  # Ball disappeared
                events.append({
                    'time': data['last_seen'],
                    'quadrant': data['current_quadrant'],
                    'color': color,
                    'type': 'Exit'
                })
                del trackers[color][center]
                if not trackers[color]:  # If this was the last ball of this color
                    del trackers[color]
            elif data['first_seen'] == frame_count:  # New ball appeared
                events.append({
                    'time': frame_count,
                    'quadrant': data['current_quadrant'],
                    'color': color,
                    'type': 'Entry'
                })
            elif data['current_quadrant'] != get_quadrant(center):  # Ball changed quadrant
                events.append({
                    'time': frame_count,
                    'quadrant': data['current_quadrant'],
                    'color': color,
                    'type': 'Exit'
                })
                data['current_quadrant'] = get_quadrant(center)
                events.append({
                    'time': frame_count,
                    'quadrant': data['current_quadrant'],
                    'color': color,
                    'type': 'Entry'
                })
    return events

In [11]:
video_path = "/content/sample.mp4"
events = process_video(video_path)

# Output events
for event in events:
    print(f"Time: {event['time']}, Quadrant: {event['quadrant']}, "
          f"Color: {event['color']}, Type: {event['type']}")

Time: 1, Quadrant: 4, Color: red, Type: Entry
Time: 1, Quadrant: 2, Color: red, Type: Entry
Time: 2, Quadrant: 2, Color: red, Type: Exit
Time: 14, Quadrant: 4, Color: red, Type: Exit
Time: 61, Quadrant: 4, Color: yellow, Type: Entry
Time: 62, Quadrant: 4, Color: yellow, Type: Entry
Time: 63, Quadrant: 4, Color: yellow, Type: Entry
Time: 63, Quadrant: 4, Color: red, Type: Entry
Time: 64, Quadrant: 4, Color: yellow, Type: Entry
Time: 64, Quadrant: 4, Color: red, Type: Entry
Time: 65, Quadrant: 2, Color: yellow, Type: Entry
Time: 65, Quadrant: 4, Color: red, Type: Entry
Time: 66, Quadrant: 2, Color: yellow, Type: Entry
Time: 61, Quadrant: 4, Color: yellow, Type: Exit
Time: 67, Quadrant: 2, Color: yellow, Type: Entry
Time: 62, Quadrant: 4, Color: yellow, Type: Exit
Time: 63, Quadrant: 4, Color: yellow, Type: Exit
Time: 64, Quadrant: 4, Color: yellow, Type: Exit
Time: 64, Quadrant: 4, Color: red, Type: Exit
Time: 65, Quadrant: 2, Color: yellow, Type: Exit
Time: 65, Quadrant: 4, Color: red, 